In [ ]:
#import libraries
import os
import pandas as pd
import re
import csv
from io import StringIO
from pathlib import Path
import numpy as np
import glob

In [ ]:
#another way to read multiple csvs and add date column
path = r'/Users/gio/Google Drive/Money/House Expenses/SM Orders/SM Orders' # use your path
all_files = glob.glob(path + "*.csv")

file_date = re.compile('\d{4}-\d{2}-\d{2}')

dfs = []
for file in all_files:
    df = pd.read_csv(file)
    df['Date'] = file_date.findall(file)[0]    
    dfs.append(df)
    
df_all = pd.concat(dfs, ignore_index=True)


In [ ]:
df = df_all

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
#remove unnecessary rows
df = df.loc[-df.item.str.contains('Item Total'),:]

In [ ]:
#remove nan
df = df.dropna()

In [ ]:
#get rid of Pesos
df.total = df.total.str.replace('₱','').astype('float64')

In [ ]:
#rename qty to qty_raw
df.rename(columns = {'qty':'qty_raw', 'item':'item_raw'}, inplace = True)

In [ ]:
#extract number to get qty
df['qty'] = df.qty_raw.str.extract(r'(\d+ )')

In [ ]:
#extract units from content
df['units'] = df.qty_raw.str.replace('(s$)','').str.replace(r'\d+ ','')
#df['units'] = df.Units.str.replace(r'(\d+)','').fillna('pcs')
#df['units'] = df.Units.replace(r'^\s*$',np.NaN, regex = True).fillna('pcs')

In [ ]:
#extract content
df['content'] = df.item_raw.str.extract('(\d+.+\n)', expand = False).str.replace('\n','')

In [ ]:
#extract item name
df['item'] = df.item_raw.str.replace(r' \S+\n.+', '')

In [ ]:
#extract unit price
df['unit_price'] = df.item_raw.str.extract(r'(₱\d+\.\d{2})', expand = False).str.replace('₱','')

In [ ]:
#change types
for col in ['total', 'qty', 'unit_price']:
    df[col] = df[col].astype('float64')

df.date = df.date.astype('datetime64')
df.units = df.units.astype('category')

In [ ]:
#lower case
df.item = df.item.str.lower()

In [ ]:
#export to csv
df.to_csv('/Users/gio/Google Drive/Money/House Expenses/SM Orders/SM Orders Logs.csv', index = None, header=True)